In [51]:
import importlib # использую для перегрузки библиотек
import time
from IPython.display import clear_output

import pandas as pd
import json
import numpy as np
import random

import data_preparing

importlib.reload(data_preparing)
clear_output(True)

In [43]:
data_path = "service_cards_tokenised_remont_only"

In [44]:
docs = data_preparing.read_and_prepared(data_path)

In [45]:
docs.shape

(854408, 12)

In [52]:
path_train, path_test = data_preparing.form_test_train_set(docs, name="simple", test_size=0.1)

In [53]:
path_castom = data_preparing.form_castom_set(docs, name="simple")

In [54]:
path_castom

'vw_castomsimple'

In [55]:
path_train

'vw_train_richsimple'